In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas_ta as ta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
import tensorflow as tf
from tensorflow import keras


In [3]:
df_nflx_intraday = pd.read_csv('data_nflx_intraday.csv', index_col=0, parse_dates=True) # will be our working df
df = df_nflx_intraday.copy() # working df
df.columns = df.columns.str.lower()

In [4]:
df.head()
df2 = df.copy()
df3 = df.copy()
df4 = df.copy()
dfx = df.copy()

In [5]:
""" # adding a few indicators
df['rsi'] = ta.rsi(df['close'], length=30)
df['ad'] = ta.ad(df['high'], df['low'], df['close'], df['volume']) # Accumulation/Distribution Line
bbands = ta.bbands(df['close'], length=10, mamode="ema") #  Returns: pd.DataFrame: lower, mid, upper, bandwidth, and percent columns.
df[['bbandsl', 'bbandsm', 'bbandsu','bbandsb', 'bbandsp']] = bbands[['BBL_10_2.0', 'BBM_10_2.0', 'BBU_10_2.0', 'BBB_10_2.0', 'BBP_10_2.0']]

# getting rid of nan
df.dropna(inplace=True) """

df.ta.strategy()


63it [00:06, 10.45it/s]


AttributeError: 'Series' object has no attribute 'append'

In [6]:
df.info()
df['price_increased'] = np.where(df['close'].diff() > 0, 1, 0)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17506 entries, 2023-07-27 04:00:00 to 2023-08-24 19:59:00
Columns: 176 entries, open to MAD_30
dtypes: float64(169), int32(7)
memory usage: 23.2 MB


In [7]:
num_columns = df.shape[1]
print("Number of columns:", num_columns)

Number of columns: 177


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17506 entries, 2023-07-27 04:00:00 to 2023-08-24 19:59:00
Columns: 177 entries, open to price_increased
dtypes: float64(169), int32(8)
memory usage: 23.2 MB


In [9]:
df.dropna(inplace=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4932 entries, 2023-07-27 05:53:00 to 2023-08-24 19:19:00
Columns: 177 entries, open to price_increased
dtypes: float64(169), int32(8)
memory usage: 6.5 MB


In [11]:
# dividing X and y
X = df.drop(columns=['close'])
y = df[['close']]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
X.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4932 entries, 2023-07-27 05:53:00 to 2023-08-24 19:19:00
Columns: 176 entries, open to price_increased
dtypes: float64(168), int32(8)
memory usage: 6.5 MB


In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)


In [15]:
num_features = X_train.shape[1]

In [16]:
# Creating model using sequential API ** PAGE 296 **
model = keras.models.Sequential([
 keras.layers.Dense(30, activation="relu", input_shape=(num_features,)),
 keras.layers.Dense(1, activation="linear")
])


In [17]:
# Compiling the model
model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(X_train, y_train, epochs=20,
           validation_split=0.1, batch_size=32 )


Epoch 1/20
111/111 [==============================] - 0s 2ms/step - loss: 295672384.0000 - val_loss: 238567.6719
Epoch 2/20
111/111 [==============================] - 0s 737us/step - loss: 53783.0391 - val_loss: 2832.8765
Epoch 3/20
111/111 [==============================] - 0s 709us/step - loss: 744.2428 - val_loss: 157.3642
Epoch 4/20
111/111 [==============================] - 0s 718us/step - loss: 140.5345 - val_loss: 124.2412
Epoch 5/20
111/111 [==============================] - 0s 746us/step - loss: 133.1657 - val_loss: 123.7476
Epoch 6/20
111/111 [==============================] - 0s 900us/step - loss: 133.0127 - val_loss: 123.7331
Epoch 7/20
111/111 [==============================] - 0s 882us/step - loss: 133.0664 - val_loss: 123.6831
Epoch 8/20
111/111 [==============================] - 0s 909us/step - loss: 133.0336 - val_loss: 123.7453
Epoch 9/20
111/111 [==============================] - 0s 855us/step - loss: 133.0442 - val_loss: 123.7332
Epoch 10/20
111/111 [===============

Conclusion: this is really good since both the losses are falling and both the accuracies are increasing. And the fact that the curves are really close to each other this means we dont have overfitting happening. 
If we are not happy with the result we can tune the 
    hyperparameters ( number of layers, number of neurons per layer, type of activation for each hidden layer, the epochs, batch siz (it can be set in the fit() method using the
batch_size argument, which defaults to 32)) 

In [18]:
# Evaluating the model
model.evaluate(X_test, y_test)

31/31 [==============================] - 0s 633us/step - loss: 132.3094


132.30938720703125

In [19]:
# making predictions
X_new = X_test[200:202]
predictions = model.predict(X_new)
predictions


1/1 [==============================] - 0s 55ms/step


array([[425.12445],
       [425.12445]], dtype=float32)

In [20]:
y_test[200:202]

,close
date,
2023-08-16 18:26:00,413.49
2023-08-21 06:57:00,406.50


In [21]:
input = keras.layers.Input(shape=X_train.shape[1:])
hidden = keras.layers.Dense(30, activation="relu")(input)
concat = keras.layers.Concatenate()([input, hidden])
output = keras.layers.Dense(1, activation="linear")(concat)
model = keras.models.Model(inputs=[input], outputs=[output])

In [22]:
X_train

array([[0.64693631, 0.59809339, 0.6770963 , ..., 0.46067734, 0.27184243,
        0.        ],
       [0.60462567, 0.56368531, 0.65045504, ..., 0.49507834, 0.07950957,
        1.        ],
       [0.73758632, 0.68107758, 0.76507078, ..., 0.36612527, 0.1329709 ,
        0.        ],
       ...,
       [0.62391757, 0.57785334, 0.66601198, ..., 0.50756758, 0.13278249,
        0.        ],
       [0.06532939, 0.06223814, 0.17110688, ..., 0.40315808, 0.07952979,
        0.        ],
       [0.80017538, 0.74989374, 0.82060905, ..., 0.15695753, 0.6088491 ,
        0.        ]])

In [23]:
model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(X_train, y_train, epochs=20,
           validation_split=0.1, batch_size=32 )

Epoch 1/20
111/111 [==============================] - 0s 2ms/step - loss: 43242.5586 - val_loss: 160.7921
Epoch 2/20
111/111 [==============================] - 0s 837us/step - loss: 104.3979 - val_loss: 99.9506
Epoch 3/20
111/111 [==============================] - 0s 873us/step - loss: 72.4741 - val_loss: 72.8248
Epoch 4/20
111/111 [==============================] - 0s 827us/step - loss: 54.3377 - val_loss: 71.9803
Epoch 5/20
111/111 [==============================] - 0s 818us/step - loss: 44.1030 - val_loss: 53.7868
Epoch 6/20
111/111 [==============================] - 0s 827us/step - loss: 37.6676 - val_loss: 44.1239
Epoch 7/20
111/111 [==============================] - 0s 827us/step - loss: 33.0172 - val_loss: 39.7652
Epoch 8/20
111/111 [==============================] - 0s 800us/step - loss: 29.6271 - val_loss: 36.5964
Epoch 9/20
111/111 [==============================] - 0s 837us/step - loss: 27.5055 - val_loss: 36.0882
Epoch 10/20
111/111 [==============================] - 0s 855

Complex models using the Functional API with multiple input

In [24]:
df_A = X.iloc[:, :4]
df_B =X.iloc[:, 4:]
num_features_A = df_A.shape[1]
num_features_B = df_B.shape[1]

In [25]:
input_A = keras.layers.Input(shape=(num_features_A,))
input_B = keras.layers.Input(shape=(num_features_B,))
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, activation="linear")(concat)
model = keras.models.Model(inputs=[input_A, input_B], outputs=[output])

In [26]:
model.compile(loss="mse", optimizer="sgd")
X_train_A, X_train_B = X_train[:, :4], X_train[:, 4:]
X_test_A, X_test_B = X_test[:, :4], X_test[:, 4:]
X_new_A, X_new_B = X_test_A[:10], X_test_B[:10]
history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
        validation_split=0.1, batch_size=32)
mse_test = model.evaluate((X_test_A, X_test_B), y_test)



Epoch 1/20
111/111 [==============================] - 0s 1ms/step - loss: nan - val_loss: nan
Epoch 2/20
111/111 [==============================] - 0s 827us/step - loss: nan - val_loss: nan
Epoch 3/20
111/111 [==============================] - 0s 837us/step - loss: nan - val_loss: nan
Epoch 4/20
111/111 [==============================] - 0s 900us/step - loss: nan - val_loss: nan
Epoch 5/20
111/111 [==============================] - 0s 882us/step - loss: nan - val_loss: nan
Epoch 6/20
111/111 [==============================] - 0s 809us/step - loss: nan - val_loss: nan
Epoch 7/20
111/111 [==============================] - 0s 818us/step - loss: nan - val_loss: nan
Epoch 8/20
111/111 [==============================] - 0s 855us/step - loss: nan - val_loss: nan
Epoch 9/20
111/111 [==============================] - 0s 827us/step - loss: nan - val_loss: nan
Epoch 10/20
111/111 [==============================] - 0s 800us/step - loss: nan - val_loss: nan
Epoch 11/20
111/111 [====================

In [27]:
y_pred = model.predict((X_new_A, X_new_B))

1/1 [==============================] - 0s 39ms/step


In [28]:
y_pred

array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan]], dtype=float32)

In [29]:
y_test[:10]

,close
date,
2023-08-22 14:40:00,413.350
2023-08-02 14:52:00,431.120
2023-08-22 19:17:00,412.935
2023-08-22 08:25:00,410.000
2023-07-27 16:36:00,412.900
2023-07-31 08:17:00,426.500
2023-08-08 12:01:00,436.740
2023-07-27 11:40:00,422.950
2023-07-31 15:10:00,437.850


Multi output neural network

In [30]:
input_A = keras.layers.Input(shape=(num_features_A,))
input_B = keras.layers.Input(shape=(num_features_B,))
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, activation="linear")(concat)

aux_output = keras.layers.Dense(1)(hidden2)
model = keras.models.Model(inputs=[input_A, input_B],
outputs=[output, aux_output])
X_train_A, X_train_B = X_train[:, :4], X_train[:, 4:]
X_test_A, X_test_B = X_test[:, :4], X_test[:, 4:]
X_new_A, X_new_B = X_test_A[:10], X_test_B[:10]
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer="sgd")
history = model.fit([X_train_A, X_train_B], [y_train, y_train], epochs=20,
  validation_split=0.1, batch_size=32)
mse_test = model.evaluate((X_test_A, X_test_B), y_test)


Epoch 1/20
111/111 [==============================] - 1s 2ms/step - loss: nan - dense_9_loss: nan - dense_10_loss: nan - val_loss: nan - val_dense_9_loss: nan - val_dense_10_loss: nan
Epoch 2/20
111/111 [==============================] - 0s 937us/step - loss: nan - dense_9_loss: nan - dense_10_loss: nan - val_loss: nan - val_dense_9_loss: nan - val_dense_10_loss: nan
Epoch 3/20
111/111 [==============================] - 0s 973us/step - loss: nan - dense_9_loss: nan - dense_10_loss: nan - val_loss: nan - val_dense_9_loss: nan - val_dense_10_loss: nan
Epoch 4/20
111/111 [==============================] - 0s 946us/step - loss: nan - dense_9_loss: nan - dense_10_loss: nan - val_loss: nan - val_dense_9_loss: nan - val_dense_10_loss: nan
Epoch 5/20
111/111 [==============================] - 0s 937us/step - loss: nan - dense_9_loss: nan - dense_10_loss: nan - val_loss: nan - val_dense_9_loss: nan - val_dense_10_loss: nan
Epoch 6/20
111/111 [==============================] - 0s 1ms/step - loss

In [31]:
total_loss, main_loss, aux_loss = model.evaluate(
 [X_test_A, X_test_B], [y_test, y_test])

31/31 [==============================] - 0s 667us/step - loss: nan - dense_9_loss: nan - dense_10_loss: nan


In [32]:
y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])


1/1 [==============================] - 0s 43ms/step


In [33]:
y_pred_main

array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan]], dtype=float32)

In [34]:
y_pred_aux

array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan]], dtype=float32)